# EEG - Flow

## 1. Convert XDF to FIFF

The Lab Streaming Layer streams (LSL) stored in the *.xdf* file must be loaded and converted into a FIFF format.
The *.xdf* file contains:
- *eegoSports 000xxx*: the EEG data stream, with `000xxx` the serial number (S/N) of the amplifier.
- *Oddball_task*: triggers of the oddball task, duplicate of the hardware triggers from received on the `TRIGGER` channel of the EEG data stream.
- *OBS_webcam* (task-UT-only): Frame IDs from the webcam OBS video stream.
- *OBS_gameplay* (task-UT-only): Frame IDs from the screen capture OBS video stream.
- *UT_GameEvents* (task-UT-only): Events from Unreal Tournament.
- *MouseButtons* (task-UT-only): Mouse clicks and button events.
- *MousePosition* (task-UT-only): Mouse X/Y position.
- *Keyboard* (task-UT-only): Keyboard button events.

Last edit: 21.06.2023 18:56

In [1]:
import multiprocessing as mp
import time
from itertools import product

from mne import set_log_level as set_log_level_mne

from eeg_flow import set_log_level
from eeg_flow.utils import parallel
from eeg_flow.tasks import convert_xdf_to_fiff

2024-04-19 16:03:08,512 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-04-19 16:03:08,514 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
set_log_level_mne("WARNING")
set_log_level("INFO")

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [9]:
PARTICIPANTS_WITH_GROUPS: list[str] = ["S24-G6"]  # list of str "Pxx-Gy", e.g. ["P02-G2"]
TASKS: list[str]                    = ["oddball","UT"]  # ["oddball"], ["UT"] or ["oddball", "UT"]
RUNS: list[int]                     = [1,2]  # [1], [2] or [1, 2]

inputs = [
    (t[0].split("-") + list(t[1:]))
    for t in product(PARTICIPANTS_WITH_GROUPS, TASKS, RUNS)
]
print(inputs)

[['S24', 'G6', 'oddball', 1], ['S24', 'G6', 'oddball', 2], ['S24', 'G6', 'UT', 1], ['S24', 'G6', 'UT', 2]]


The variable `inputs` contains is a list of list. Each sublist defines one file by its participant, group, task and run attribute. Each sublist is one set of input variable for `convert_xdf_to_fiff` which will be picked up by a worker (process) and executed. For each execution, the created derivatives are:
- RAW recording (`_raw.fif`)
- Oddball task annotations (`_oddball_annot.fif`)
- Additional stream annotations (keyboard, mouse, .. only if task is `UT`, `_stream_annot.fif`)

In [10]:
%%time
current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

assert len(inputs) != 0  # sanity-check
n_jobs = min(len(inputs), mp.cpu_count() - 2)
parallel(convert_xdf_to_fiff, n_jobs, inputs)

Start time 16:25:47


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:42<00:00, 25.54s/it]

CPU times: total: 78.1 ms
Wall time: 1min 42s


[None, None, None, None]